## 01. 라이브러리 로드

In [ ]:
import pandas as pd
import numpy as np

## 02. 데이터 불러오기 및 정제

In [2]:
df = pd.read_csv("data/krx/KOSPI_KOSDAQ_log_returns_2011_2024.csv", encoding='utf-8-sig')

In [3]:
df['날짜'] = pd.to_datetime(df['날짜'])
df.set_index('날짜', inplace=True)

## 03. 연간변동성 계산
- 날짜가 어떤 기간에 속하는지 라벨링 ex) 2014년 3월은 2013년 4월말 시작 기간에 속함
- 실제 기간 시작일을 매핑
- 기간 시작일과 티커로 그룹화 후 계산
- 결과 확인 후 데이터프레임으로 변환
- 시작일에 1년을 더해 종료일 생성
- 컬럼명 재정의

In [4]:
print("최적화된 계산 시작...")

# 각 날짜가 어떤 '기간'에 속하는지 라벨링합니다.
period_year = np.where(df.index.month > 4, df.index.year, df.index.year - 1)
df['period_year'] = period_year

# 각 period_year에 해당하는 실제 '기간 시작일'을 매핑합니다.
period_start_dates = {
    year: pd.to_datetime(f'{year}-04-01') + pd.offsets.BMonthEnd()
    for year in range(df['period_year'].min(), df['period_year'].max() + 1)
}
df['period_start_date'] = df['period_year'].map(period_start_dates)


# groupby를 사용해 '기간 시작일'과 '티커'로 그룹화 후 한 번에 계산
annual_volatility = (
    df.groupby(['period_start_date', '티커'])['로그수익률']
    .std()
    * np.sqrt(252)
)

print("계산 완료.")

# --- 최종 결과 확인 ---
results_df = annual_volatility.reset_index()
results_df.rename(columns={'로그수익률': '연간변동성'}, inplace=True)
results_df.rename(columns={'period_start_date': '시작일'}, inplace=True)

# 시작일에 1년을 더해 종료일 생성 (표시 목적)
results_df['종료일'] = results_df['시작일'].apply(
    lambda date: pd.to_datetime(f'{date.year + 1}-04-01') + pd.offsets.BMonthEnd()
)

results_df = results_df[['시작일', '종료일', '티커', '연간변동성']]

최적화된 계산 시작...
계산 완료.


In [ ]:
results_df['회계년도'] = results_df['시작일'].dt.year
results_df.drop(columns=['시작일', '종료일'], inplace=True)
results_df.sort_values(by=['티커', '회계년도'], inplace=True)

## 04. 데이터 저장

In [ ]:
results_df.to_excel("data/krx/연간변동성.xlsx", index=False)